In [1]:
backend = "np_backend"

In [2]:
import time
import numpy as np

np.random.seed(0)

match backend:
    case "original_backend":
        from lib.original_backend.linear_algebra import Matrix
        from lib.original_backend.nn import  NN, ReLU, Linear, LayerNorm
        from lib.original_backend.processing import ColumnNormalizer
        from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

    case "np_backend":
        from lib.np_backend.linear_algebra import Matrix
        from lib.np_backend.nn import NN, ReLU, Linear, LayerNorm
        from lib.np_backend.processing import ColumnNormalizer
        from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

    case "pt_backend":
        from lib.pt_backend.linear_algebra import Matrix
        from lib.pt_backend.nn import NN, ReLU, Linear
        from lib.pt_backend.processing import ColumnNormalizer
        from lib.pt_backend.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

from lib.metrics.losses import mean_squared_error
from lib.gd_data_loaders import BatchDataLoader, StochasticDataLoader, MiniBatchDataLoader

In [3]:
# Boston House Price Dataset
# The Boston House Price Dataset involves the prediction of a house price in thousands of dollars given details of the house and its neighborhood.
# It is a regression problem. There are 506 observations with 13 input variables and 1 output variable. The variable names are as follows:

data = []
with open("data/boston_house_prices.data", "rt") as f:
    for line in f.readlines():
        data.append([float(v) for v in line.split()])
data = np.array(data)

In [4]:
np.random.shuffle(data)
split = int(len(data) * 0.8)
train_set = data[:split]
test_set = data[split:]

X_train, y_train = train_set[:, :-1], train_set[:, -1:]
X_test, y_test = test_set[:, :-1], test_set[:, -1:]
X_train = Matrix(X_train)
y_train = Matrix(y_train)
X_test = Matrix(X_test)
y_test = Matrix(y_test)
X_train.dims(), y_train.dims(), X_test.dims(), y_test.dims()

((404, 13), (404, 1), (102, 13), (102, 1))

In [5]:
normalizer = ColumnNormalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)
X_train.dims(), X_test.dims()

((404, 13), (102, 13))

In [6]:
def init_nn():
    return NN([
        Linear(13, 4),
        LayerNorm(4),
        ReLU(),
        Linear(4, 1),
    ])

In [7]:
time_point = time.time()

data_loaders = [
    # BatchDataLoader(X_train, y_train),
    # StochasticDataLoader(X_train, y_train),
    MiniBatchDataLoader(X_train, y_train, 32)
]
optimizer_creators = [
    # lambda nn: SgdOptimizer(nn, 0.001),
    # lambda nn: SgdWithMomentumOptimizer(nn, 0.001, 0.9),
    lambda nn: AdaGradOptimizer(nn, 0.05),
    lambda nn: RmsPropOptimizer(nn, 0.01, 0.95),
    lambda nn: AdamOptimizer(nn, 0.01, 0.95, 0.95),
]

for data_loader in data_loaders:
    for optimizer_creator in optimizer_creators:
        nn = init_nn()
        optimizer = optimizer_creator(nn)
        print(f"gradient descent: {data_loader.__class__} | optimizer: {optimizer.__class__}")
        for i in range(2001):
            X_b, y_b = data_loader.get_batch()
            out = nn(X_b)
            loss = mean_squared_error(y_b, out)
            if i % 100 == 0:
                elapsed_time = int(time.time() - time_point)
                time_point = time.time()
                print(f"{i} | {loss.data:.2f} | {elapsed_time}s") 

            optimizer.step(loss)

        train_out = nn(X_train) 
        train_loss = mean_squared_error(y_train, train_out)
        test_out = nn(X_test) 
        test_loss = mean_squared_error(y_test, test_out)
        print(f"train loss: {train_loss.data:.2f}   test loss: {test_loss.data:.2f}") 

gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.AdaGradOptimizer'>
0 | 779.13 | 0s
100 | 252.22 | 15s
200 | 284.72 | 16s
300 | 138.75 | 16s
400 | 116.39 | 4s
500 | 55.73 | 16s
600 | 37.97 | 16s
700 | 24.84 | 15s
800 | 45.33 | 15s
900 | 89.13 | 15s
1000 | 32.70 | 15s
1100 | 72.07 | 4s
1200 | 38.83 | 15s
1300 | 65.42 | 16s
1400 | 20.61 | 16s
1500 | 32.34 | 16s
1600 | 44.64 | 16s
1700 | 55.61 | 16s
1800 | 33.22 | 4s
1900 | 36.65 | 15s
2000 | 51.23 | 16s
train loss: 45.84   test loss: 48.46
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.RmsPropOptimizer'>
0 | 432.49 | 0s
100 | 215.62 | 16s
200 | 39.92 | 16s
300 | 32.53 | 17s
400 | 41.55 | 16s
500 | 23.06 | 4s
600 | 37.85 | 16s
700 | 26.67 | 17s
800 | 29.93 | 18s
900 | 12.17 | 17s
1000 | 51.05 | 18s
1100 | 31.85 | 4s
1200 | 17.32 | 16s
1300 | 32.93 | 17s
1400 | 13.84 | 17s
1500 | 22.99 | 17s
1600 | 68.29 | 17s
1700 | 36.92 | 4s
1

In [8]:
[(round(float(v1),1), float(v2)) for v1, v2 in zip([v[0].data for v in out], [v[0].data for v in y_test])][:10]

[(23.9, 20.7),
 (18.7, 39.8),
 (21.4, 17.8),
 (18.2, 19.6),
 (22.6, 14.9),
 (19.3, 22.0),
 (21.6, 48.8),
 (39.5, 25.0),
 (13.7, 48.5),
 (20.0, 23.9)]

In [9]:
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.AdaGradOptimizer'>
0 | 800.47 | 0s
100 | 146.44 | 14s
200 | 89.14 | 3s
300 | 73.32 | 14s
400 | 31.86 | 15s
500 | 19.64 | 3s
600 | 21.66 | 14s
700 | 46.49 | 15s
800 | 12.77 | 14s
900 | 19.35 | 3s
1000 | 33.66 | 15s
1100 | 14.49 | 16s
1200 | 58.57 | 3s
1300 | 14.34 | 15s
1400 | 17.35 | 16s
1500 | 25.67 | 3s
1600 | 13.69 | 16s
1700 | 37.87 | 16s
1800 | 28.68 | 3s
1900 | 56.13 | 15s
2000 | 23.63 | 13s
train loss: 20.74   test loss: 21.01
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.RmsPropOptimizer'>
0 | 501.73 | 0s
100 | 131.49 | 13s
200 | 30.43 | 3s
300 | 41.33 | 13s
400 | 31.23 | 13s
500 | 9.93 | 2s
600 | 12.77 | 13s
700 | 10.20 | 14s
800 | 15.21 | 3s
900 | 10.28 | 16s
1000 | 14.14 | 16s
1100 | 27.00 | 3s
1200 | 25.46 | 16s
1300 | 6.39 | 16s
1400 | 7.05 | 3s
1500 | 11.76 | 15s
1600 | 10.33 | 15s
1700 | 15.61 | 14s
1800 | 8.01 | 3s
1900 | 9.71 | 15s
2000 | 10.79 | 16s
train loss: 11.84   test loss: 8.42
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.AdamOptimizer'>
0 | 760.70 | 0s
100 | 128.24 | 2s
200 | 39.71 | 40043s
300 | 18.63 | 17s
400 | 35.28 | 3s
500 | 12.50 | 18s
600 | 8.23 | 15s
700 | 20.96 | 2s
800 | 24.46 | 14s
900 | 9.09 | 14s
1000 | 25.91 | 2s
1100 | 13.78 | 14s
1200 | 5.10 | 14s
1300 | 6.54 | 2s
1400 | 18.26 | 14s
1500 | 19.59 | 17s
1600 | 11.90 | 2s
1700 | 11.78 | 16s
1800 | 14.75 | 16s
1900 | 10.77 | 3s
2000 | 10.37 | 15s
train loss: 12.84   test loss: 12.21

SyntaxError: invalid decimal literal (2114980610.py, line 2)